In [1]:
'''
Name   : Rupesh Garsondiya
github : @Rupeshgarsonidya
Topic  : Convolutional Neural Network (CNN) with torch
'''

'\nName   : Rupesh Garsondiya\ngithub : @Rupeshgarsonidya\nTopic  : Convolutional Neural Network (CNN) with torch \n'

# **CNN Using PyTorch**

**Reference :** PyTroch playlist By Nitish Sir (Campus X)

**Video Link :** https://www.youtube.com/watch?v=hkiBZLRFvO4&t=602s

**About This notebook :** You can gain practical knowledge on how to build a convolutional neural network using Torch.


**prerequisites :**Understanding of CNN , Basic of torch

**Suggestion :**If you want to understand how convolutional neural networks work, check out my deep learning repository. In that repo, I explain all deep learning concepts in detail with both theoretical and practical guides. Check it out!

In [2]:
# import required library

import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [3]:
# set ramdom seed for reproducibility
torch.manual_seed(42)

In [4]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


!kaggle datasets download -d zalando-research/fashionmnist

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/zalando-research/fashionmnist
License(s): other
fashionmnist.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:

import zipfile
zipfile = zipfile.ZipFile('/content/fashionmnist.zip')
zipfile.extractall('/content')
zipfile.close()

In [6]:
# check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [7]:

df_train = pd.read_csv('/content/fashion-mnist_train.csv')
df_train.head()

df_test = pd.read_csv('/content/fashion-mnist_test.csv')
df_test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:

# Extract features and labels for training set
# X_train: All columns except 'label' from the df_train DataFrame are used as features for the training set
# y_train: The 'label' column from df_train DataFrame is used as the target labels for the training set
X_train = df_train.drop('label', axis=1).values
y_train = df_train['label'].values

# Extract features and labels for test set
# X_test: All columns except 'label' from the df_test DataFrame are used as features for the test set
# y_test: The 'label' column from df_test DataFrame is used as the target labels for the test set
X_test = df_test.drop('label', axis=1).values
y_test = df_test['label'].values

# Print the shape (dimensions) of the training and testing data
# This helps confirm the structure and size of the datasets
print('X_train : ', X_train.shape)  # Prints the shape of the training features
print('y_train : ', y_train.shape)  # Prints the shape of the training labels
print('X_test : ', X_test.shape)    # Prints the shape of the test features
print('y_test : ', y_test.shape)    # Prints the shape of the test labels

X_train :  (60000, 784)
y_train :  (60000,)
X_test :  (10000, 784)
y_test :  (10000,)


In [9]:
# Define a custom dataset class inheriting from PyTorch's Dataset class
class CustomDataset(Dataset):

  # Initialization method (__init__) to store the features and targets
  # features: Input features of the dataset (e.g., image data, tabular data)
  # targets: Corresponding labels or target values for the features
  def __init__(self, features, targets):
    # Convert features and targets to torch tensors with appropriate data types
    self.features = torch.tensor(features, dtype=torch.float32).reshape(-1,1,28,28) # This give a an 1-D tensor but in CNN work with 2-D that why here we apply reshape on the feature
    self.targets = torch.tensor(targets, dtype=torch.long)  # Targets are stored as long (integer) tensors

  # Method to return the number of samples in the dataset
  def __len__(self):
    return self.features.shape[0]  # The length is equal to the number of rows in the features array

  # Method to retrieve a sample by index
  # idx: Index of the data sample to retrieve
  def __getitem__(self, idx):
    return self.features[idx], self.targets[idx]  # Return the features and targets for the given index

In [27]:
# create train dataset object
train_dataset = CustomDataset(X_train,y_train)



# create test dataset object
test_dataset = CustomDataset(X_test,y_test)

train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)


# **No changes until this code. chnage only in architecture code**

In [45]:
# Define a CNN architechture

class MYCNN(nn.Module):
  def __init__(self,input_feature):
    super().__init__()

    self.features = nn.Sequential(
        nn.Conv2d(in_channels=input_feature,out_channels=32,kernel_size=3,stride=1,padding='same'), # padding same it mean after convolutional image size is same
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(kernel_size=2,stride=2),
        nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding='same'),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=64*7*7,out_features=128),
        nn.Dropout(0.4),
        nn.ReLU(),

        nn.Linear(in_features=128,out_features=10),



    )

  def forward(self,x):
    x = self.features(x)
    x = self.classifier(x)
    return x

In [49]:
learning_rate = 0.01
epochs = 5

In [50]:
model = MYCNN(1)

model.to(device=device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [57]:
# Training Loop

# Iterate through the specified number of epochs
for epochs in range(epochs):

    # Initialize variable to track total loss for this epoch
    total_epochs_loss = 0

    # Iterate through each batch of data in the train_loader (this handles batching and shuffling of the data)
    for batch_feature, label in train_loader:

        batch_feature = batch_feature.to(device)
        label = label.to(device)

        # Forward pass: Pass the batch of features through the model to get the predictions (output)
        output = model(batch_feature)

        # Calculate the loss between the predicted output and true labels
        loss = criterion(output, label)

        # Add the current loss to the total loss for the epoch
        total_epochs_loss += loss.item()

        # Backward pass: Compute gradients of the loss with respect to model parameters
        optimizer.zero_grad()  # Clear the previous gradients
        loss.backward()        # Backpropagate the loss to compute new gradients

        # Update model parameters (weights and biases) using the optimizer
        optimizer.step()

    # Calculate the average loss for the epoch (divide by the number of batches in the train_loader)
    avg_loss = total_epochs_loss / len(train_loader)

    # Print the average loss for the current epoch
    print(f'Epoch : {epochs+1} Loss : {avg_loss}')


Epoch : 1 Loss : 0.2780778299406171
Epoch : 2 Loss : 0.26836161455164353
Epoch : 3 Loss : 0.2782452471673489


In [52]:

# set model to eval mode
# we need to define our model explicitly you use as evalulate beacuse in deep leaarning some time behave diffrent during training and testing
# like dropout we fropout apply on only during traing we not apply dropout on testing
# same in case of the batch normalization


model.eval()

MYCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): Dropout(p=0.4, inplace=False)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [55]:
# Evaluation code: Evaluates the model on the test set

# Initialize variables to track the total number of samples and number of correct predictions
total = 0
correct = 0

# Disable gradient calculation to save memory and computation during evaluation
# `torch.no_grad()` ensures that gradients are not calculated during this phase
with torch.no_grad():

  # Iterate through each batch in the test_loader (this handles batching of test data)
  for batch_feature, label in test_loader:

    batch_feature = batch_feature.to(device)
    label = label.to(device)

    # Forward pass: Pass the batch of test features through the model to get the predictions (output)
    output = model(batch_feature)

    # Get the predicted class by selecting the index with the highest output score for each sample
    # `torch.max(output, 1)` returns the maximum value and its index along dimension 1 (the classes)
    _, predicted = torch.max(output, 1)

    # Update the total number of samples
    total = total + label.size(0)

    # Update the number of correct predictions by comparing predicted classes with actual labels
    correct = correct + (predicted == label).sum().item()

# Calculate and print the accuracy as the ratio of correct predictions to the total number of samples
print(f'Accuracy : {correct/total}')


Accuracy : 0.8874
